In [85]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
import json
import os 
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler
import joblib  # 用於保存模型

In [86]:
label_to_coordinates = {
    "1-1": (0, 0), "1-2": (0.6, 0), "1-3": (1.2, 0), "1-4": (1.8, 0), "1-5": (2.4, 0), "1-6": (3.0, 0),"1-7": (3.6, 0), "1-8": (4.2, 0), "1-9": (4.8, 0), "1-10": (5.4, 0), "1-11": (6.0, 0),
    "2-1": (0, 0.6), "2-11": (6.0, 0.6),
    "3-1": (0, 1.2), "3-11": (6.0, 1.2),
    "4-1": (0, 1.8), "4-11": (6.0, 1.8),
    "5-1": (0, 2.4), "5-11": (6.0, 2.4),
    "6-1": (0, 3.0), "6-2": (0.6, 3.0), "6-3": (1.2, 3.0), "6-4": (1.8, 3.0), "6-5": (2.4, 3.0),"6-6": (3.0, 3.0), "6-7": (3.6, 3.0), "6-8": (4.2, 3.0), "6-9": (4.8, 3.0), "6-10": (5.4, 3.0), "6-11": (6.0, 3.0),
    "7-1": (0, 3.6), "7-11": (6.0, 3.6),
    "8-1": (0, 4.2), "8-11": (6.0, 4.2),
    "9-1": (0, 4.8), "9-11": (6.0, 4.8),
    "10-1": (0, 5.4), "10-11": (6.0, 5.4),
    "11-1": (0, 6.0), "11-2": (0.6, 6.0), "11-3": (1.2, 6.0), "11-4": (1.8, 6.0), "11-5": (2.4, 6.0),"11-6": (3.0, 6.0), "11-7": (3.6, 6.0), "11-8": (4.2, 6.0), "11-9": (4.8, 6.0), "11-10": (5.4, 6.0), "11-11": (6.0, 6.0)
}
label_mapping = {
    '11': '1-1','10': '1-2','9': '1-3','8': '1-4','7': '1-5','6': '1-6','5': '1-7','4': '1-8','3': '1-9','2': '1-10','1': '1-11',
    '12': '2-1','30': '2-11',
    '13': '3-1','29': '3-11',
    '14': '4-1','28': '4-11',
    '15': '5-1','27': '5-11',
    '16': '6-1','17': '6-2','18': '6-3','19': '6-4','20': '6-5','21': '6-6','22': '6-7','23': '6-8','24': '6-9','25': '6-10','26': '6-11',
    '49': '7-1','31': '7-11',
    '48': '8-1','32': '8-11',
    '47': '9-1','33': '9-11',
    '46': '10-1','34': '10-11',
    '45': '11-1','44': '11-2','43': '11-3','42': '11-4','41': '11-5','40': '11-6','39': '11-7','38': '11-8','37': '11-9','36': '11-10','35': '11-11'
}

In [87]:
selected_columns = ['Label',
                        'AP2_Distance (mm)','AP4_Distance (mm)',
                        'AP2_StdDev (mm)','AP4_StdDev (mm)',
                                'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi'
                                ]  

what_data = "test"
# what_data = "DNN_3Layer_1AP_Best_combine_F_S_R_20241214_usingearlystopepchoc10000"

# 'AP1_Distance (mm)',
# 'AP1_StdDev (mm)',

In [88]:
# 1. 讀取資料
file_path = "timestamp_allignment_Balanced_2024_12_14_rtt_logs.csv"

date = "2024_12_14"

data = pd.read_csv(file_path, usecols=selected_columns)

print(data.head())

target_column = 'Label'  # 替換成目標欄位名稱
label_column = 'Label'  # 替換成目標欄位名稱

  Label  AP2_Distance (mm)  AP4_Distance (mm)  AP1_Rssi  AP2_Rssi  AP3_Rssi  \
0  1-11             8178.0             4852.0     -60.0     -68.0     -61.0   
1  1-11             8139.0             4911.0     -62.0     -70.0     -59.0   
2  1-11             8139.0             4647.0     -61.0     -69.0     -62.0   
3  1-11             8022.0             6219.0     -62.0     -69.0     -63.0   
4  1-11             8335.0             4471.0     -61.0     -70.0     -62.0   

   AP4_Rssi  AP2_StdDev (mm)  AP4_StdDev (mm)  
0     -59.0           1502.0            934.0  
1     -59.0           1466.0            485.0  
2     -55.0             91.0            600.0  
3     -55.0           1523.0           2649.0  
4     -55.0           1195.0           1079.0  


In [89]:
# 資料前處理 (一): 刪除前後n筆資料
n = 1
# 確保依據Label排序
data = data.sort_values(by=label_column).reset_index(drop=True)

# 建立一個空的 DataFrame 用於存放處理後的資料
processed_data = pd.DataFrame(columns=data.columns)


# # 針對每個Label群組進行處理
# for label, group in data.groupby(label_column):
#     group = group.iloc[n:-n]
#     # 將處理後的群組資料加入
#     processed_data = pd.concat([processed_data, group], ignore_index=True)

processed_data = data
processed_data


,Label,AP2_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP2_StdDev (mm),AP4_StdDev (mm)
0,1-1,6606.0,-538.0,-62.0,-73.0,-65.0,-48.0,1513.0,1125.0
1,1-1,6811.0,-811.0,-51.0,-73.0,-67.0,-55.0,1239.0,1318.0
2,1-1,6841.0,-850.0,-63.0,-73.0,-67.0,-56.0,1262.0,1503.0
3,1-1,6841.0,-733.0,-53.0,-73.0,-67.0,-55.0,1265.0,1350.0
4,1-1,6811.0,-743.0,-58.0,-73.0,-67.0,-56.0,1275.0,1421.0
...,...,...,...,...,...,...,...,...,...
19644,9-11,7866.0,4500.0,-57.0,-69.0,-51.0,-65.0,910.0,278.0
19645,9-11,8100.0,4998.0,-58.0,-70.0,-52.0,-65.0,832.0,1038.0
19646,9-11,8061.0,4461.0,-57.0,-69.0,-51.0,-64.0,878.0,1585.0
19647,9-11,6030.0,4774.0,-57.0,-69.0,-52.0,-66.0,1142.0,145.0


In [90]:
data_imputed = processed_data.groupby(label_column).apply(
    lambda group: group.fillna(group.mean())
).reset_index()
data_imputed

print("Number of data per RP : " + str(len(data_imputed)/49))

Number of data per RP : 401.0


In [91]:
# 建立 Label 映射
y = data_imputed[target_column]
# label_mapping = {str(i): label for i, label in enumerate(y.unique())}
reverse_label_mapping = {v: int(k) - 1 for k, v in label_mapping.items()}  # 讓數字標籤 -1
y_numeric = y.map(reverse_label_mapping)

print("Final reverse_label_mapping in DNN:", reverse_label_mapping)
print("y_numeric unique values in DNN:", y_numeric.unique())

Final reverse_label_mapping in DNN: {'1-1': 10, '1-2': 9, '1-3': 8, '1-4': 7, '1-5': 6, '1-6': 5, '1-7': 4, '1-8': 3, '1-9': 2, '1-10': 1, '1-11': 0, '2-1': 11, '2-11': 29, '3-1': 12, '3-11': 28, '4-1': 13, '4-11': 27, '5-1': 14, '5-11': 26, '6-1': 15, '6-2': 16, '6-3': 17, '6-4': 18, '6-5': 19, '6-6': 20, '6-7': 21, '6-8': 22, '6-9': 23, '6-10': 24, '6-11': 25, '7-1': 48, '7-11': 30, '8-1': 47, '8-11': 31, '9-1': 46, '9-11': 32, '10-1': 45, '10-11': 33, '11-1': 44, '11-2': 43, '11-3': 42, '11-4': 41, '11-5': 40, '11-6': 39, '11-7': 38, '11-8': 37, '11-9': 36, '11-10': 35, '11-11': 34}
y_numeric unique values in DNN: [10  1  0  9  8  7  6  5  4  3  2 45 33 44 35 34 43 42 41 40 39 38 37 36
 11 29 12 28 13 27 14 26 15 24 25 16 17 18 19 20 21 22 23 48 30 47 31 46
 32]


In [92]:
# 把label部分拿掉
X = data_imputed.drop(columns=['level_1','Label'])
print(X.head())

   AP2_Distance (mm)  AP4_Distance (mm)  AP1_Rssi  AP2_Rssi  AP3_Rssi  \
0             6606.0             -538.0     -62.0     -73.0     -65.0   
1             6811.0             -811.0     -51.0     -73.0     -67.0   
2             6841.0             -850.0     -63.0     -73.0     -67.0   
3             6841.0             -733.0     -53.0     -73.0     -67.0   
4             6811.0             -743.0     -58.0     -73.0     -67.0   

   AP4_Rssi  AP2_StdDev (mm)  AP4_StdDev (mm)  
0     -48.0           1513.0           1125.0  
1     -55.0           1239.0           1318.0  
2     -56.0           1262.0           1503.0  
3     -55.0           1265.0           1350.0  
4     -56.0           1275.0           1421.0  


In [93]:
scaler = StandardScaler()
columns_to_scale = selected_columns.copy()  # 建立副本，避免影響原始變數
columns_to_scale.remove('Label')  # 在副本上移除 'Label'
X_scaled = scaler.fit_transform(X[columns_to_scale])

# 保存標準化器
joblib.dump(scaler, f'scaler_{what_data}.pkl')

print(X_scaled)

[[ 0.4715099  -1.33155776  1.15459382 ... -1.06008268 -1.29455783
   1.50052306]
 [ 0.54164566 -1.44425472  0.73590891 ... -1.06008268 -1.67424054
   0.4157486 ]
 [ 0.55190943 -1.46035429  0.77105399 ... -1.06008268 -1.67424054
   0.26078082]
 ...
 [ 0.96930273  0.73207635  0.1842839  ... -0.32852003  1.36322117
  -0.97896143]
 [ 0.27444552  0.86128568  0.58768834 ... -0.32852003  1.17337981
  -1.28889699]
 [ 0.94261693  0.81298699 -0.26496195 ... -0.32852003  1.17337981
  -1.28889699]]


In [94]:
from sklearn.model_selection import StratifiedShuffleSplit

In [95]:
ap = 'test'
root = 'test'

dataamount = 320
N_val = 20

N_train = dataamount # 訓練集每個類別至少要有 N_train 筆資料
test_val_ratio = 1  # 剩餘資料中，50% 作為驗證集，50% 作為測試集

In [96]:
# 轉為 DataFrame 方便操作
data = pd.DataFrame(X_scaled)
data['label'] = y_numeric  # 加入 label 欄位

train_data_full = data.groupby('label', group_keys=False).sample(n=N_train, replace=False,random_state=42) 

train_data_full


,0,1,2,3,4,5,6,7,label
1087,1.036359,0.832802,0.926915,0.167039,-0.335343,-0.328520,-0.535192,-0.204123,0
1083,0.982646,0.832802,1.085832,-0.134654,-0.122245,-0.511411,-0.914875,-0.359090,0
835,0.982646,0.699877,0.702292,0.563811,-0.122245,-0.511411,-0.535192,-0.359090,0
1013,0.995989,0.869129,1.142369,-0.112713,-0.122245,-0.328520,-0.725034,-0.204123,0
895,0.995989,0.966139,1.143897,-0.862374,-0.122245,-0.328520,-0.725034,-0.204123,0
...,...,...,...,...,...,...,...,...,...
17392,-0.968497,-0.799859,0.719100,-0.743525,-0.974635,0.585933,-0.345351,-0.049155,48
17482,-1.021869,-0.848158,0.928443,-0.633819,-1.400830,0.585933,-0.535192,-0.204123,48
17401,-1.142297,-0.912556,0.981924,-0.546054,-1.187732,0.585933,-0.725034,-0.204123,48
17590,-1.088583,-0.799859,1.030822,-0.745354,-0.974635,0.585933,-0.535192,-0.359090,48


In [97]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=N_val / N_train,random_state=42) 
train_index, val_index = next(sss.split(train_data_full.drop(columns=['label']), train_data_full['label']))
train_data = train_data_full.iloc[train_index]
val_data = train_data_full.iloc[val_index]

remaining_data = data.drop(train_data_full.index)

In [98]:
X_train, y_train = train_data.drop(columns=['label']).values, train_data['label'].values
X_val, y_val = val_data.drop(columns=['label']).values, val_data['label'].values
X_test, y_test = remaining_data.drop(columns=['label']).values, remaining_data['label'].values

# **計算每個 Set 內各 Label 的資料數量**
train_label_counts = pd.Series(y_train).value_counts().sort_index()
val_label_counts = pd.Series(y_val).value_counts().sort_index()
test_label_counts = pd.Series(y_test).value_counts().sort_index()

# **確保所有 Labels 都有出現在三個 Set 裡**
all_labels = sorted(set(train_label_counts.index) | set(val_label_counts.index) | set(test_label_counts.index))
label_distribution = pd.DataFrame(index=all_labels)

label_distribution["Training Set"] = train_label_counts
label_distribution["Validation Set"] = val_label_counts
label_distribution["Test Set"] = test_label_counts

# **用 0 填補缺失值（表示該 Label 在該 Set 中沒有數據）**
label_distribution = label_distribution.fillna(0).astype(int)

from IPython.display import display
display(label_distribution)

,Training Set,Validation Set,Test Set
0,300,20,81
1,300,20,81
2,300,20,81
3,300,20,81
4,300,20,81
5,300,20,81
6,300,20,81
7,300,20,81
8,300,20,81
9,300,20,81


In [99]:
import time
# 記錄開始時間
start_time = time.time()

In [100]:
# X_train, y_train 
# X_val, y_val
# X_test, y_test 

In [101]:
# 建立 DNN 模型
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(len(label_mapping), activation='softmax')
])

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 設定 EarlyStopping 回呼函數
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 訓練模型，包含驗證集
model.fit(X_train, y_train,
          validation_data=(X_val, y_val),
          epochs=10000, batch_size=32, verbose=1, callbacks=[early_stop])

Epoch 1/10000
460/460 [==============================] - 1s 1ms/step - loss: 1.0259 - accuracy: 0.7778 - val_loss: 0.2167 - val_accuracy: 0.9561
Epoch 2/10000
460/460 [==============================] - 1s 2ms/step - loss: 0.1643 - accuracy: 0.9637 - val_loss: 0.1570 - val_accuracy: 0.9724
Epoch 3/10000
460/460 [==============================] - 1s 1ms/step - loss: 0.1118 - accuracy: 0.9735 - val_loss: 0.1479 - val_accuracy: 0.9745
Epoch 4/10000
460/460 [==============================] - 1s 1ms/step - loss: 0.0906 - accuracy: 0.9793 - val_loss: 0.1275 - val_accuracy: 0.9776
Epoch 5/10000
460/460 [==============================] - 1s 2ms/step - loss: 0.0750 - accuracy: 0.9820 - val_loss: 0.1320 - val_accuracy: 0.9816
Epoch 6/10000
460/460 [==============================] - 1s 1ms/step - loss: 0.0667 - accuracy: 0.9831 - val_loss: 0.1222 - val_accuracy: 0.9816
Epoch 7/10000
460/460 [==============================] - 1s 1ms/step - loss: 0.0575 - accuracy: 0.9841 - val_loss: 0.1194 - val_ac

In [ ]:
# 記錄結束時間
end_time = time.time()


# Needsave
# 計算訓練時間（秒）
training_time = end_time - start_time
print(f"訓練時間：{training_time:.2f} 秒")

訓練時間：11.12 秒


In [ ]:
mde_report_per_fold = {}

# 進行預測
y_test_pred_numeric = model.predict(X_test)
y_pred_classes = np.argmax(y_test_pred_numeric, axis=1)
y_test_pred_labels = [label_mapping[str(num + 1)] for num in y_pred_classes]  # 補回 +1


# 計算 MDE
y_test_pred_coordinates = np.array([label_to_coordinates[label] for label in y_test_pred_labels])
y_test_coordinates = np.array([label_to_coordinates[label_mapping[str(label + 1)]] for label in y_test])


distances = np.linalg.norm(y_test_pred_coordinates - y_test_coordinates, axis=1)
avg_mde = np.mean(distances)
print(f"MDE: {avg_mde:.4f}")

# 記錄每個 RP 在當前 fold 的 MDE
for true_label, distance in zip(y_test, distances):
    if true_label not in mde_report_per_fold:
        mde_report_per_fold[true_label] = []
    mde_report_per_fold[true_label].append(distance)  # 存所有 fold 的 MDE

# 計算 5-Fold 平均 MDE
mde_report_avg = {label: {"mde": np.mean(distances), "count": len(distances)} for label, distances in mde_report_per_fold.items()}

# 儲存到 JSON 檔案
file_path = "Testing_mde"
with open(file_path, "w") as f:
    json.dump(mde_report_avg, f, indent=4)

print(f"MDE report saved to: {file_path}")


  1/125 [..............................] - ETA: 1s

125/125 [==============================] - 0s 626us/step
0.036737949435162606
